In [7]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

from glob import glob

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
training_set = train_datagen.flow_from_directory('Classification/train/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 28045 images belonging to 17 classes.


In [10]:
class_indices = training_set.class_indices

inv_class_indices = {v: k for k, v in class_indices.items()}


In [1]:
inv_class_indices = {0: 'Ambulance',
 1: 'Barge',
 2: 'Bicycle',
 3: 'Boat',
 4: 'Bus',
 5: 'Car',
 6: 'Cart',
 7: 'Caterpillar',
 8: 'Helicopter',
 9: 'Limousine',
 10: 'Motorcycle',
 11: 'Segway',
 12: 'Snowmobile',
 13: 'Tank',
 14: 'Taxi',
 15: 'Truck',
 16: 'Van'}

In [2]:
import cv2
import numpy as np
import time
import easyocr
from tensorflow.keras.models import load_model

reader = easyocr.Reader(['en'])

detected_vehicles = {'numberplates': [], 'class_labels': [], 'speeds': [], 'counts': []}


# Load pre-trained Haar Cascade classifier
plate_cascade = cv2.CascadeClassifier('NUM_PLATE.xml')

# cap = cv2.VideoCapture('vehicle-counting.mp4')
cap = cv2.VideoCapture('WhatsApp Video 2023-05-02 at 01.42.18.mp4')
# cap = cv2.VideoCapture('vehicle-counting.mp4')

# initialize variables
prev_frame = None
prev_time = None
min_width_rect = 10 # min width rectangle
min_height_rect = 10 # min height rectangle
count_line_position = 325
# Initialize Subtractor
# algo = cv2.bgsegm.createBackgroundSubtractorMOG()
car_count = 0
# Load car detection classifier
car_classifier = cv2.CascadeClassifier('cars.xml')

def center_handle(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy
# Initialize Subtractor
algo = cv2.bgsegm.createBackgroundSubtractorMOG()

detect = []
offset = 4# allowable offset
counter = 0


# Load the trained model
model = load_model('vehicle_VGG19.h5')

labels = ['Ambulance', 'Bus', 'Car', 'Limousine', 'Motorcycle', 'Tank', 'Taxi', 'Truck', 'Van']

car_cascade = cv2.CascadeClassifier('cars.xml')

while True:
    ret, frame1 = cap.read()
    if not ret:
        break
    plate_text = 'Can\'t read'
    frame1 = cv2.resize(frame1, (640, 480))
    frame1_copy = frame1.copy()
    frame = frame1.copy()

    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)
    cars = car_cascade.detectMultiScale(grey, 1.1, 3)


    for (x, y, w, h) in cars:
        car_img = frame[y:y+h, x:x+w]
        car_img = cv2.resize(car_img, (224, 224))
        car_img = cv2.cvtColor(car_img, cv2.COLOR_BGR2RGB)
        car_img = car_img / 255.0
        car_img = np.expand_dims(car_img, axis=0)
        
        pred = model.predict(car_img)[0]
        predicted_labels = inv_class_indices[np.argmax(pred)] 
        
        
        if predicted_labels in labels:
            
            cv2.rectangle(frame1_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame1_copy, predicted_labels, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # store the first frame and time
    if prev_frame is None:
        prev_frame = blur
        prev_time = time.time()
        continue

    # calculate the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, blur)
    # applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, hierarchy = cv2.findContours(dilatada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw counting line
    # cv2.line(frame1_copy, (25, count_line_position), (1208, count_line_position), (255, 127, 0), 3)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        # validate size of detected object
        validate_counter = (w >= min_width_rect) and (h >= min_height_rect)
        if not validate_counter:
            continue

        # calculate the speed of the car
        car_roi = grey[y-10:y+h+10, x-10:x+w+10]
        car_rects = car_classifier.detectMultiScale(grey, 1.1, 3)

        for (car_x, car_y, car_w, car_h) in car_rects:
            try:
                plates = plate_cascade.detectMultiScale(grey, 1.3, 5)
                # Loop through detected number plates
                for (x, y, w, h) in plates:
                    # Crop number plate from frame
                    plate_img = grey[y:y + h, x:x + w]

                    # Apply thresholding to enhance characters in number plate
                    thresh = cv2.threshold(plate_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

                    #  read the number plate text
                    plate_text = reader.readtext(thresh, paragraph=False)

                    # Draw bounding box around number plate and display number plate text
                    cv2.rectangle(frame1_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    try:

                        cv2.putText(frame1_copy, str(plate_text[0][1]), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    except:
                        cv2.putText(frame1_copy, "Can't read", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            


            except:
                print("a")

            if car_h > 20:
                curr_time = time.time()
                time_diff = curr_time - prev_time
                speed = (car_w / time_diff) * 0.0268224  # conversion factor from pixels per second to miles per hour
                prev_frame = blur
                prev_time = curr_time
            if speed > 50 : 
                cv2.putText(frame1_copy, f"Speed: {speed:.2f} mph - OVERSPEEDING", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.putText(frame1_copy, f"Speed: {speed:.2f} mph", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            # Draw rectangle around detected object
            cv2.rectangle(frame1_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # check if the center of the bounding box crossed the counting line
        center_y = int(y + h / 2)
        if center_y >= count_line_position and center_y <= count_line_position + 5:
            car_count += 1
            try: 
                if predicted_labels and car_count and speed and str(plate_text[0][1]):
                    detected_vehicles['numberplates'].append(str(plate_text[0][1]))
                    detected_vehicles['class_labels'].append(predicted_labels)
                    detected_vehicles['counts'].append(car_count)
                    detected_vehicles['speeds'].append(speed)
            except:
                pass

    # Display car count
    cv2.putText(frame1_copy, f"Car count: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    


    # Display the resulting frame
    cv2.imshow('Traffic Analysis', frame1_copy)
    if cv2.waitKey(1) == ord('q'):
        break
#Release the capture and close the window
cap.release()
cv2.destroyAllWindows()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


1/1 [==============================] - 0s 379ms/step


In [3]:
from fpdf import FPDF

# create the PDF object
pdf = FPDF()
pdf.add_page()

# set the font and font size
pdf.set_font('Arial', 'B', 16)

# create the table header
pdf.cell(50, 10, 'Number Plate', border=1)
pdf.cell(50, 10, 'Class Label', border=1)
pdf.cell(30, 10, 'Speed', border=1)
pdf.cell(30, 10, 'Count', border=1)
pdf.ln()

# loop through the detected_vehicles dictionary and add the data to the table
for i in range(len(detected_vehicles['numberplates'])):
    pdf.cell(50, 10, str(detected_vehicles['numberplates'][i]), border=1)
    pdf.cell(30, 10, str(detected_vehicles['class_labels'][i]), border=1)
    pdf.cell(30, 10, str(detected_vehicles['speeds'][i]), border=1)
    pdf.cell(30, 10, str(detected_vehicles['counts'][i]), border=1)
    pdf.ln()

# save the PDF file
pdf.output('detected_vehicles_report.pdf', 'F')


''